In [3]:
# v2
# 使用说明
# 1、头条的格式必须是【cpsid+素材+日期】。例如【2102556+xxxxx+0415】
#   - 日期需为cpsid开始投放的日期。
#     逻辑：相同cpsid计划的日期不可以有重叠。比如4月15日新建了一条cpsid为2102551的计划，如果当天有2102551的计划正在投放中
# 2、素材的格式必须是【日期-素材制作者-素材名】。例如【220420-毛亚兰-如何打老郭】
# 3、除指定位置，文字中间请不要使用【-+】这两个符号。
#    常见问题：
#      - 素材名：220223-毛亚兰-武尊刷本+ui音效2【竖版】。中间不要使用【+】
#      - cpsid 2102555R,21025555爆R,21025555-R都不可以，仅可以使用21025555


import pandas as pd
import re
import numpy as np

pd.options.display.max_columns=None
pd.options.display.max_rows=None

data_a = pd.read_excel(r'.\toutiao1.xls')
data_b = pd.read_excel(r'.\yunchan.xlsx')
file_a = pd.DataFrame()
file_b = pd.DataFrame()
cpsid_loc = 'cpsid-self.txt'
cpid_loc = 'cpid.txt'
A_COMSUME_THRETHOLD = 0


def get_cpsid(id_loc):
    cpsid = []
    txt = open(id_loc, 'r').readlines()
    for line in txt:
        line = line.replace('\n', '')
        cpsid.append(line)
    return cpsid


# filtering unneeded yunchan data
def filter_data(data):
    print('start')
    tongjifangshi = ['TD']

    print('readlines')
    cpid = get_cpsid(cpid_loc)

    print('filtering 统计方式')
    data = data[data['统计方式'].isin(tongjifangshi)]

    data['广告商'] = data['广告商'].astype(str)
    print('filtering cpid')
    data = data[data['广告商'].isin(cpid)]

    print('filtering 直播数据')
    data['广告id'] = data['广告id'].map(str)
    data['日期'] = data['日期'].map(str)
    data = data.drop(data[data['广告id'].str.contains(pat=r'^2103.*', regex=True)].index)
    data = data.drop(data[data['广告id'].str.contains(pat=r'^2111.*', regex=True)].index)
    data = data.drop(data[data['广告id'].str.contains(pat=r'^2133.*', regex=True)].index)

    return data


# return pivot table of correspnding dataframe
def get_pt(pt):
    new_pt = pd.DataFrame()
    new_pt['消耗'] = pt['消耗']
    new_pt['激活成本'] = round(pt['消耗'] / pt['激活数'], 1)
    new_pt['付费成本'] = round(pt['消耗'] / pt['首日付费人数'], 1)
    new_pt['点击率（%）'] = round(pt['点击数'] / pt['曝光数'], 4) * 100
    new_pt['转化率（%）'] = round(pt['激活数'] / pt['点击数'], 3) * 100
    new_pt['注册率（%）'] = round(pt['注册数'] / pt['激活数'], 3) * 100
    new_pt['首日付费率（%）'] = round(pt['首日付费人数'] / pt['激活数'], 3) * 100
    new_pt['次留率（%）'] = round(pt['次留数'] / pt['激活数'], 3) * 100
    new_pt['首日ROI（%）'] = round(pt['首日付费金额'] / pt['消耗'], 3) * 100
    new_pt['三日ROI（%）'] = round(pt['F3付费金额'] / pt['消耗'], 3) * 100
    new_pt['3/1涨幅'] = round(new_pt['三日ROI（%）'] / new_pt['首日ROI（%）'], 2)
    new_pt['有效率（%）'] = round(pt['有效数'] / pt['激活数'], 3) * 100
    return new_pt

def get_sorted(pre_sort):
    sorted_df = pd.DataFrame()
    sorted_df['id']=pre_sort['id']
    sorted_df['cpsid']=pre_sort['cpsid_x']
    sorted_df['日期']=pre_sort['日期_x']
    sorted_df['素材名']=pre_sort['素材名']
    sorted_df['消耗']=pre_sort['消耗']
    sorted_df['曝光数']=pre_sort['曝光数']
    sorted_df['点击数']=pre_sort['点击数']
    sorted_df['25%进度播放数']=pre_sort['25%进度播放数']
    sorted_df['50%进度播放数']=pre_sort['50%进度播放数']
    sorted_df['75%进度播放数']=pre_sort['75%进度播放数']
    sorted_df['99%进度播放数']=pre_sort['99%进度播放数']
    sorted_df['有效播放数']=pre_sort['有效播放数']
    sorted_df['激活数']=pre_sort['激活数']
    sorted_df['注册数']=pre_sort['注册数']
    sorted_df['次留数']=pre_sort['次留数']
    sorted_df['首日付费人数']=pre_sort['首日付费人数']
    sorted_df['首日付费金额']=pre_sort['首日付费金额']
    sorted_df['F3付费金额']=pre_sort['F3付费金额']
    sorted_df['自然周付费金额']=pre_sort['自然周付费金额']
    sorted_df['有效数']=pre_sort['有效数']
    sorted_df['广告计划'] = pre_sort['广告计划']
    return sorted_df
    

# file A: front-end data， plan in tiktok

file_a['cpsid'] = data_a['广告计划'].str.split("+", expand=True)[0]
file_a['日期'] = [x[2:10] for x in data_a['时间']]
file_a['日期'] = file_a['日期'].str.replace('-', '')
file_a['素材名'] = data_a['广告计划'].str.split('+', expand=True)[1].str.split('-', expand=True)[2]
file_a['消耗'] = data_a['消耗']
file_a['曝光数'] = data_a['展示数']
file_a['点击数'] = data_a['点击数']

file_a['25%进度播放数'] = data_a['25%进度播放数']
file_a['50%进度播放数'] = data_a['50%进度播放数']
file_a['75%进度播放数'] = data_a['75%进度播放数']
file_a['99%进度播放数'] = data_a['99%进度播放数']
file_a['有效播放数'] = data_a['有效播放数']
file_a['广告计划'] = data_a['广告计划']
file_a['id'] = file_a['cpsid'] + file_a['日期']
# target_toutiao = file_a[file_a['cpsid'].isin(cpsid)]

file_a = file_a[file_a['消耗'] > A_COMSUME_THRETHOLD]
file_a = file_a.sort_values(by=['cpsid', '日期'], ascending=True)

# file B: back-end data，yunchan
# when a.cpsid=b.cpsid put in file b

#data_b = filter_data(data_b)
#print(data_b)
data_b = data_b.replace('-', 0)
data_b.fillna(0, inplace=True)
file_b['日期'] = [x[2:10] for x in data_b['日期']]
file_b['日期'] = file_b['日期'].str.replace('-', '')
file_b['cpsid'] = data_b['广告id']
file_b['激活数']=data_b['激活']
file_b['注册数'] = data_b['注册']
file_b['次留数'] = data_b['次留']
file_b['首日付费人数'] = data_b['c1']
file_b['首日付费金额'] = data_b['f1']
file_b['自然周付费金额'] = data_b['至今付费']
file_b['F3付费金额'] = data_b['f3'].apply(float)
file_b['有效数'] = data_b['有效激活设备数']
file_b['次留'] = data_b['次留']

# cpsid=get_cpsid(cpsid_loc)
# file_b = file_b[file_b['cpsid'].map(str).isin(cpsid)]
file_b['id'] = file_b['cpsid'].map(str) + file_b['日期']

pd_merge = pd.merge(file_a, file_b, on='id', how='inner')
pd_merge_groupby=pd_merge[['id','消耗']].groupby(by='id',as_index=False).max()
pd_merge_groupby=pd.merge(pd_merge_groupby,pd_merge,on=['id','消耗'],how='inner')
sorted_pd_merge_groupby=get_sorted(pd_merge_groupby)

#print(pd_merge_groupby)
pt_merge = pd.pivot_table(pd_merge, index=[u'素材名'], aggfunc=np.sum, margins=True)
new_pt_merge = get_pt(pt_merge)

writer = pd.ExcelWriter('output.xlsx')

file_a.to_excel(writer, 'toutiao-all', index=False)
file_b.to_excel(writer, 'yunchan-all', index=False)
pd_merge.to_excel(writer, 'merge', index=False)
pd_merge_groupby.to_excel(writer, 'pd_merge_groupby', index=False)
sorted_pd_merge_groupby.to_excel(writer, 'sorted_pd_merge_groupby', index=False)
pt_merge.to_excel(writer, 'pt_merge')
new_pt_merge.to_excel(writer, 'new_pt_merge')

writer.save()

print("done")


done


In [ ]:
# v3
# 使用说明
# 1、头条的格式必须是【cpsid+素材+日期】。例如【2102556+xxxxx+0415】
#   - 日期需为cpsid开始投放的日期。
#     逻辑：相同cpsid计划的日期不可以有重叠。比如4月15日新建了一条cpsid为2102551的计划，如果当天有2102551的计划正在投放中
# 2、素材的格式必须是【日期-素材制作者-素材名】。例如【220420-毛亚兰-如何打老郭】
# 3、除指定位置，文字中间请不要使用【-+】这两个符号。
#    常见问题：
#      - 素材名：220223-毛亚兰-武尊刷本+ui音效2【竖版】。中间不要使用【+】
#      - cpsid 2102555R,21025555爆R,21025555-R都不可以，仅可以使用21025555


import pandas as pd
import re
import numpy as np

pd.options.display.max_columns=None
pd.options.display.max_rows=None

data_a = pd.read_excel(r'.\reyun.xls')
data_b = pd.read_excel(r'.\v3.xlsx')
file_a = pd.DataFrame()
file_b = pd.DataFrame()
cpsid_loc = 'cpsid-self.txt'
cpid_loc = 'cpid.txt'
A_COMSUME_THRETHOLD = 0


def get_cpsid(id_loc):
    cpsid = []
    txt = open(id_loc, 'r').readlines()
    for line in txt:
        line = line.replace('\n', '')
        cpsid.append(line)
    return cpsid


# return pivot table of correspnding dataframe
def get_pt(pt):
    new_pt = pd.DataFrame()
    new_pt['消耗'] = pt['消耗']
    new_pt['激活成本'] = round(pt['消耗'] / pt['激活数'], 1)
    new_pt['付费成本'] = round(pt['消耗'] / pt['首日付费人数'], 1)
    new_pt['点击率（%）'] = round(pt['点击数'] / pt['曝光数'], 4) * 100
    new_pt['转化率（%）'] = round(pt['激活数'] / pt['点击数'], 3) * 100
    new_pt['注册率（%）'] = round(pt['注册数'] / pt['激活数'], 3) * 100
    new_pt['首日付费率（%）'] = round(pt['首日付费人数'] / pt['激活数'], 3) * 100
    new_pt['次留率（%）'] = round(pt['次留数'] / pt['激活数'], 3) * 100
    new_pt['首日ROI（%）'] = round(pt['首日付费金额'] / pt['消耗'], 3) * 100
    new_pt['三日ROI（%）'] = round(pt['F3付费金额'] / pt['消耗'], 3) * 100
    new_pt['3/1涨幅'] = round(new_pt['三日ROI（%）'] / new_pt['首日ROI（%）'], 2)
    new_pt['有效率（%）'] = round(pt['有效数'] / pt['激活数'], 3) * 100
    return new_pt

def get_sorted(pre_sort):
    sorted_df = pd.DataFrame()
    sorted_df['id']=pre_sort['id']
    sorted_df['cpsid']=pre_sort['cpsid_x']
    sorted_df['日期']=pre_sort['日期_x']
    sorted_df['素材名']=pre_sort['素材名']
    sorted_df['消耗']=pre_sort['消耗']
    sorted_df['曝光数']=pre_sort['曝光数']
    sorted_df['点击数']=pre_sort['点击数']
    sorted_df['25%进度播放数']=pre_sort['25%进度播放数']
    sorted_df['50%进度播放数']=pre_sort['50%进度播放数']
    sorted_df['75%进度播放数']=pre_sort['75%进度播放数']
    sorted_df['99%进度播放数']=pre_sort['99%进度播放数']
    sorted_df['有效播放数']=pre_sort['有效播放数']
    sorted_df['激活数']=pre_sort['激活数']
    sorted_df['注册数']=pre_sort['注册数']
    sorted_df['次留数']=pre_sort['次留数']
    sorted_df['首日付费人数']=pre_sort['首日付费人数']
    sorted_df['首日付费金额']=pre_sort['首日付费金额']
    sorted_df['F3付费金额']=pre_sort['F3付费金额']
    sorted_df['自然周付费金额']=pre_sort['自然周付费金额']
    sorted_df['有效数']=pre_sort['有效数']
    sorted_df['广告计划'] = pre_sort['广告计划']
    return sorted_df
    

# file A: front-end data， plan in tiktok

file_a['cpsid'] = data_a['广告计划'].str.split("+", expand=True)[0]
file_a['日期'] = [x[2:10] for x in data_a['时间']]
file_a['日期'] = file_a['日期'].str.replace('-', '')
file_a['素材名'] = data_a['广告计划'].str.split('+', expand=True)[1].str.split('-', expand=True)[2]
file_a['消耗'] = data_a['消耗']
file_a['曝光数'] = data_a['展示数']
file_a['点击数'] = data_a['点击数']

file_a['25%进度播放数'] = data_a['25%进度播放数']
file_a['50%进度播放数'] = data_a['50%进度播放数']
file_a['75%进度播放数'] = data_a['75%进度播放数']
file_a['99%进度播放数'] = data_a['99%进度播放数']
file_a['有效播放数'] = data_a['有效播放数']
file_a['广告计划'] = data_a['广告计划']
file_a['id'] = file_a['cpsid'] + file_a['日期']
# target_toutiao = file_a[file_a['cpsid'].isin(cpsid)]

file_a = file_a[file_a['消耗'] > A_COMSUME_THRETHOLD]
file_a = file_a.sort_values(by=['cpsid', '日期'], ascending=True)

# file B: back-end data，yunchan
# when a.cpsid=b.cpsid put in file b

#data_b = filter_data(data_b)
#print(data_b)
data_b = data_b.replace('-', 0)
data_b.fillna(0, inplace=True)
file_b['日期'] = [x[2:10] for x in data_b['日期']]
file_b['日期'] = file_b['日期'].str.replace('-', '')
file_b['cpsid'] = data_b['广告id']
file_b['激活数']=data_b['激活']
file_b['注册数'] = data_b['注册']
file_b['次留数'] = data_b['次留']
file_b['首日付费人数'] = data_b['c1']
file_b['首日付费金额'] = data_b['f1']
file_b['自然周付费金额'] = data_b['至今付费']
file_b['F3付费金额'] = data_b['f3'].apply(float)
file_b['有效数'] = data_b['有效激活设备数']
file_b['次留'] = data_b['次留']

# cpsid=get_cpsid(cpsid_loc)
# file_b = file_b[file_b['cpsid'].map(str).isin(cpsid)]
file_b['id'] = file_b['cpsid'].map(str) + file_b['日期']

pd_merge = pd.merge(file_a, file_b, on='id', how='inner')
pd_merge_groupby=pd_merge[['id','消耗']].groupby(by='id',as_index=False).max()
pd_merge_groupby=pd.merge(pd_merge_groupby,pd_merge,on=['id','消耗'],how='inner')
sorted_pd_merge_groupby=get_sorted(pd_merge_groupby)

#print(pd_merge_groupby)
pt_merge = pd.pivot_table(pd_merge, index=[u'素材名'], aggfunc=np.sum, margins=True)
new_pt_merge = get_pt(pt_merge)

writer = pd.ExcelWriter('output.xlsx')

file_a.to_excel(writer, 'toutiao-all', index=False)
file_b.to_excel(writer, 'yunchan-all', index=False)
pd_merge.to_excel(writer, 'merge', index=False)
pd_merge_groupby.to_excel(writer, 'pd_merge_groupby', index=False)
sorted_pd_merge_groupby.to_excel(writer, 'sorted_pd_merge_groupby', index=False)
pt_merge.to_excel(writer, 'pt_merge')
new_pt_merge.to_excel(writer, 'new_pt_merge')

writer.save()

print("done")
